In [73]:
participant_id = "P1"

In [74]:
%load_ext autoreload
%autoreload 2
import importlib.util
from pathlib import Path
import os
import sys
from utils.utils import *

VID_FILE_PATH = MAIN_DIR + "/data/raw/videos"
OUTPUT_CSV_FILE = MAIN_DIR + "/data/processed/interview_features.csv"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from utils.face_analyzer import FaceAnalyzer

face_analyzer = FaceAnalyzer()
frames = face_analyzer.get_video_frames_for_participant(
    participant_id, VID_FILE_PATH, num_selected_frames=25)

# Lexical Features

In [ ]:
# from src.utils.LexicalAnalyser import LexicalAnalyser

# AUDIO_FILE_PATH = f"../../data/raw/audio/trimmed_{participant_id}.wav"
# lexical_analyser = LexicalAnalyser(AUDIO_FILE_PATH)
# # Extract all features
# lexical_features = lexical_analyser.extract_all_features()

# # Print the extracted features
# print(lexical_features)

# Facial Features

### Face Mesh

In [ ]:
for frame in frames:
    detected_faces_landmarks = face_analyzer.process_image_results(frame.image)
    frame.facial_landmarks_obj= face_analyzer.get_largest_face_landmarks_obj(frame.image, detected_faces_landmarks)
    if frame.facial_landmarks_obj:
        frame.facial_landmarks = frame.facial_landmarks_obj.landmark

### Face

In [ ]:

for frame in frames:
    if frame.facial_landmarks:
        frame.face = face_analyzer.get_face_coordinates(frame.facial_landmarks, frame.image)

### Smile

In [ ]:
import numpy as np

SMOOTH_WINDOW = 5
happiness_buffer = []
def smooth_happiness(happiness_prob):
    if happiness_prob is None:
        return 0 # TODO: change?
    happiness_buffer.append(happiness_prob)
    if len(happiness_buffer) > SMOOTH_WINDOW:
        happiness_buffer.pop(0)
    return np.mean(happiness_buffer)


for i, frame in enumerate(frames):
    face_roi = face_analyzer.get_face_roi_image(frame.image, frame.face, expand_ratio=1.1)
    frame.smile = smooth_happiness(face_analyzer.get_smile_from_frame(face_roi))

Action: emotion: 100%|██████████| 1/1 [00:00<00:00,  5.49it/s]


### Selected Facial Features

In [ ]:
for frame in frames:
    frame.two_landmarks_connectors = face_analyzer.get_selected_facial_landmarks(frame.facial_landmarks)

### Head Pose

In [ ]:
texts = []
for frame in frames:
    result = face_analyzer.get_face_angles(frame.image, frame.facial_landmarks)
    frame.face_angles = result

# Prosodic Extraction

In [ ]:
from schemas.model_features import ProsodicFeatures
from utils.prosody_analyzer import ProsodyAnalyzer
 

prosody_analyzer = ProsodyAnalyzer(participant_id)
prosodic_features: ProsodicFeatures = prosody_analyzer.extract_all_features()
print(prosodic_features)

# Features Storage

### Facial Features Aggregation

In [ ]:
from utils.feature_storage import FeatureStorage


feature_storage = FeatureStorage(OUTPUT_CSV_FILE)
facial_features = feature_storage.aggregate_facial_features(frames)

In [ ]:
feature_storage.save_to_csv(participant_id, facial_features, prosodic_features)

Feature attributes: {'average_outer_brow_height_mean': 0.016292900807397526, 'average_inner_brow_height_mean': 0.016716919456475587, 'eye_open_mean': 0.010468848209014529, 'outer_lip_height_mean': 0.018316188260899348, 'inner_lip_height_mean': 0.0015409458352216691, 'lip_corner_distance_mean': 0.0440151939977027, 'smile_mean': 10.256666666666668, 'pitch_mean': -4.712518372137046, 'yaw_mean': 3.0347174116061266, 'roll_mean': 6.353102566850139, 'average_outer_brow_height_std': 0.0012285257356963754, 'average_inner_brow_height_std': 0.0020765351662113078, 'eye_open_std': 0.002062652254566344, 'outer_lip_height_std': 0.002059701403088427, 'inner_lip_height_std': 0.0010092618753403165, 'lip_corner_distance_std': 0.0019574266099187283, 'smile_std': 11.420198480470178, 'pitch_std': 3.3918804800803084, 'yaw_std': 7.172205755936284, 'roll_std': 4.98795219956107, 'average_outer_brow_height_min': 0.014478190489122942, 'average_inner_brow_height_min': 0.014040496248951373, 'eye_open_min': 0.007808

# Display Frames

In [ ]:
# for frame in frames:
#     frame.reset_drawable_image()
#     # frame.draw_face_border()
    
#     frame.draw_selected_facial_landmarks(draw_lines=True)
    
#     frame.put_face_angles()
#     # frame.draw_facial_landmarks()
#     frame.display()